In [1]:
import requests
import pandas as pd
from bs4 import BeautifulSoup

In [46]:
import threading
from datetime import datetime, date, time

def setInterval(interval, times = -1):
    # This will be the actual decorator,
    # with fixed interval and times parameter
    def outer_wrap(function):
        # This will be the function to be
        # called
        def wrap(*args, **kwargs):
            stop = threading.Event()

            # This is another function to be executed
            # in a different thread to simulate setInterval
            def inner_wrap():
                i = 0
                while i != times and not stop.isSet():
                    stop.wait(interval)
                    today = datetime.today()
                    if today.weekday() < 5 and today.hour > 10 and today.hour < 20:
                        function()
                        i += 1

            t = threading.Timer(0, inner_wrap)
            t.daemon = True
            t.start()
            return stop
        return wrap
    return outer_wrap

In [3]:
response = requests.get('https://finance.yahoo.com/gainers?offset=0&count=50')
soup = BeautifulSoup(response.text)

In [36]:
data = {
    'symbols': [],
    'name': [],
    'price': [],
    'change': [],
    'change_in_percents': []
}

for elem in soup.find_all(class_='simpTblRow'):
    content = elem.contents
    change_in_percents = float(content[4].find('span').text.replace('%', ''))
    if change_in_percents > 5 and change_in_percents < 10:
        data['symbols'].append(content[0].find('a').text)
        data['name'].append(content[1].text)
        data['price'].append(content[2].find('span').text)
        data['change'].append(content[3].find('span').text)
        data['change_in_percents'].append(content[4].find('span').text)
df = pd.DataFrame(data=data)

In [37]:
df

,symbols,name,price,change,change_in_percents
0,VIR,"Vir Biotechnology, Inc.",64.54,+5.81,+9.89%
1,WEICY,"Weichai Power Co., Ltd.",23.39,+2.09,+9.81%
2,SIG,Signet Jewelers Limited,40.62,+3.62,+9.78%
3,SI,Silvergate Capital Corporation,93.06,+8.29,+9.78%
4,FNMAI,Federal National Mortgage Association,5.0000,+0.4300,+9.41%
5,CYDY,CytoDyn Inc.,5.55,+0.45,+8.82%
6,MRNA,"Moderna, Inc.",173.16,+13.61,+8.53%
7,TWNK,"Hostess Brands, Inc.",15.35,+1.08,+7.57%
8,ERIC,Telefonaktiebolaget LM Ericsson (publ),12.47,+0.84,+7.22%
9,WDC,Western Digital Corporation,56.43,+3.78,+7.18%


In [38]:
import json

body = {
    'data': df.to_json(orient='records')
}
body

{'data': '[{"symbols":"VIR","name":"Vir Biotechnology, Inc.","price":"64.54","change":"+5.81","change_in_percents":"+9.89%"},{"symbols":"WEICY","name":"Weichai Power Co., Ltd.","price":"23.39","change":"+2.09","change_in_percents":"+9.81%"},{"symbols":"SIG","name":"Signet Jewelers Limited","price":"40.62","change":"+3.62","change_in_percents":"+9.78%"},{"symbols":"SI","name":"Silvergate Capital Corporation","price":"93.06","change":"+8.29","change_in_percents":"+9.78%"},{"symbols":"FNMAI","name":"Federal National Mortgage Association","price":"5.0000","change":"+0.4300","change_in_percents":"+9.41%"},{"symbols":"CYDY","name":"CytoDyn Inc.","price":"5.55","change":"+0.45","change_in_percents":"+8.82%"},{"symbols":"MRNA","name":"Moderna, Inc.","price":"173.16","change":"+13.61","change_in_percents":"+8.53%"},{"symbols":"TWNK","name":"Hostess Brands, Inc.","price":"15.35","change":"+1.08","change_in_percents":"+7.57%"},{"symbols":"ERIC","name":"Telefonaktiebolaget LM Ericsson (publ)","pri

In [60]:
response = requests.post('https://formfor.site/send/BwrgbwSdFR5ciVznvjjaCXC8E9QWwu', 
                         {
    'data': df.to_json(orient='records')
}, 
                        )
response.reason

'OK'

In [64]:
@setInterval(1800)
def send_data():
    response = requests.get('https://finance.yahoo.com/gainers?offset=0&count=50')
    soup = BeautifulSoup(response.text)
    
    data = {
    'symbols': [],
    'name': [],
    'price': [],
    'change': [],
    'change_in_percents': []
    }

    for elem in soup.find_all(class_='simpTblRow'):
        content = elem.contents
        change_in_percents = float(content[4].find('span').text.replace('%', ''))
        if change_in_percents > 5 and change_in_percents < 10:
            data['symbols'].append(content[0].find('a').text)
            data['name'].append(content[1].text)
            data['price'].append(content[2].find('span').text)
            data['change'].append(content[3].find('span').text)
            data['change_in_percents'].append(content[4].find('span').text)
    df = pd.DataFrame(data=data)
    print(df)
    response = requests.post('https://formfor.site/send/BwrgbwSdFR5ciVznvjjaCXC8E9QWwu', {'data': df.to_json(orient='records')})

In [65]:
stopper = send_data('bar')

   symbols                                    name   price   change  \
0      VIR                 Vir Biotechnology, Inc.   64.54    +5.81   
1    WEICY                 Weichai Power Co., Ltd.   23.39    +2.09   
2      SIG                 Signet Jewelers Limited   40.62    +3.62   
3       SI          Silvergate Capital Corporation   93.06    +8.29   
4    FNMAI   Federal National Mortgage Association  5.0000  +0.4300   
5     CYDY                            CytoDyn Inc.    5.55    +0.45   
6     MRNA                           Moderna, Inc.  173.16   +13.61   
7     TWNK                    Hostess Brands, Inc.   15.35    +1.08   
8     ERIC  Telefonaktiebolaget LM Ericsson (publ)   12.47    +0.84   
9      WDC             Western Digital Corporation   56.43    +3.78   
10   ESALY                         Eisai Co., Ltd.   79.92    +5.30   
11   ERIXF  Telefonaktiebolaget LM Ericsson (publ)   12.54    +0.82   
12    NKTR                     Nektar Therapeutics   19.70    +1.26   
13    

In [63]:
stopper.set()

   symbols                                    name   price   change  \
0      VIR                 Vir Biotechnology, Inc.   64.54    +5.81   
1    WEICY                 Weichai Power Co., Ltd.   23.39    +2.09   
2      SIG                 Signet Jewelers Limited   40.62    +3.62   
3       SI          Silvergate Capital Corporation   93.06    +8.29   
4    FNMAI   Federal National Mortgage Association  5.0000  +0.4300   
5     CYDY                            CytoDyn Inc.    5.55    +0.45   
6     MRNA                           Moderna, Inc.  173.16   +13.61   
7     TWNK                    Hostess Brands, Inc.   15.35    +1.08   
8     ERIC  Telefonaktiebolaget LM Ericsson (publ)   12.47    +0.84   
9      WDC             Western Digital Corporation   56.43    +3.78   
10   ESALY                         Eisai Co., Ltd.   79.92    +5.30   
11   ERIXF  Telefonaktiebolaget LM Ericsson (publ)   12.54    +0.82   
12    NKTR                     Nektar Therapeutics   19.70    +1.26   
13    

In [4]:
df = pd.DataFrame(data=[{'symbol': 'AMD', 'average': 1}, {'symbol': 'UAL', 'average': 0}, {'symbol': 'MU', 'average': 0}, {'symbol': 'AAPL', 'average': 0}, {'symbol': 'TSM', 'average': 1},{'symbol': 'KO', 'average': 11},])
requests.post('https://formfor.site/send/BwrgbwSdFR5ciVznvjjaCXC8E9QWwu', {'data': df.to_json(orient='records')})

<Response [200]>